In [7]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')

In [8]:
from Utils.utils import create_URM, create_ICM, combine_matrices, write_submission, create_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


In [9]:
URM = create_URM()
ICM = create_ICM()

In [15]:
URM_train_validation,URM_test = split_train_in_two_percentage_global_sample(URM,train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation,train_percentage=0.85)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13635 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13627 ( 0.2%) Users that have less than 1 test interactions


In [11]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='warp')
model.fit(URM_train, epochs=10)

train_precision = precision_at_k(model, URM_train, k=10).mean()
test_precision = precision_at_k(model, URM_test, k=10).mean()

train_auc = auc_score(model, URM_train).mean()
test_auc = auc_score(model, URM_test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.63, test 0.11.
AUC: train 0.93, test 0.91.


In [20]:
## In order to evaluate put it in a recommender class
from src.Recommenders.Base.BaseRecommender import BaseRecommender
from lightfm import LightFM
import numpy as np

class LightFMCFRecommender(BaseRecommender):
    """LightFMCFRecommender"""

    RECOMMENDER_NAME = "LightFMCFRecommender"

    def __init__(self, URM_train):
        super(LightFMCFRecommender, self).__init__(URM_train)


    def fit(self, epochs = 300, alpha = 1e-6, n_factors = 30, n_threads = 4):
        
        # Let's fit a WARP model
        self.lightFM_model = LightFM(loss='warp',
                                     item_alpha=alpha,
                                     no_components=n_factors)

        self.lightFM_model = self.lightFM_model.fit(URM_train, 
                                       epochs=epochs,
                                       num_threads=n_threads)


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        
        # Create a single (n_items, ) array with the item score, then copy it for every user
        items_to_compute = np.arange(self.n_items)
        
        item_scores = - np.ones((len(user_id_array), self.n_items)) * np.inf

        for user_index, user_id in enumerate(user_id_array):
            item_scores[user_index] = self.lightFM_model.predict(int(user_id), 
                                                                 items_to_compute)

        return item_scores

In [21]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
hyperparameters_range_dictionary = {
    "epochs": Integer(10,500),
    "alpha": Real(1e-6,0.1),
    "n_threads": Categorical([10])
}
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

tuning_class = SearchBayesianSkopt(recommender_class=LightFMCFRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 50
n_random_starts = n_cases*0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize= 10,
                    n_cases= n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=LightFMCFRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 36, 'alpha': 0.014320670446157282, 'n_threads': 10}
LightFMCFRecommender: URM Detected 1 ( 0.0%) items with no interactions.
EvaluatorHoldout: Processed 13627 (100.0%) in 21.25 sec. Users per second: 641
SearchBayesianSkopt: New best config found. Config 0: {'epochs': 36, 'alpha': 0.014320670446157282, 'n_threads': 10} - results: PRECISION: 0.0002789, PRECISION_RECALL_MIN_DEN: 0.0002862, RECALL: 0.0000568, MAP: 0.0000693, MAP_MIN_DEN: 0.0000704, MRR: 0.0006931, NDCG: 0.0002609, F1: 0.0000943, HIT_RATE: 0.0027886, ARHR_ALL_HITS: 0.0006931, NOVELTY: 0.0097618, AVERAGE_POPULARITY: 0.0035180, DIVERSITY_MEAN_INTER_LIST: 0.6583253, DIVERSITY_HERFINDAHL: 0.9658277, COVERAGE_ITEM: 0.0048175, COVERAGE_ITEM_CORRECT: 0.0011629, COVERAGE_USER: 0.9983150, COVERAGE_USER_CORRECT: 0.0027839, DIVERSITY_GINI: 0.0015383, SHANNON_ENTROPY: 5.1394563, RATIO_DIVERSITY_HERFINDAHL: 0.9662001, RATIO_DIV

Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 530, in _objective_function
    recommender_instance.save_model(self.output_folder_path, file_name =self.output_file_name_root + "_best_model")
  File "/Users/gabriele/PycharmProjects/RecSys/src/Recommenders/Base/BaseRecommender.py", line 221, in save_model
    raise NotImplementedError("BaseRecommender: save_model not implemented")
NotImplementedError: BaseRecommender: save_model not implemented


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 191.8968
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 359, 'alpha': 0.05939570163937069, 'n_threads': 10}
LightFMCFRecommender: URM Detected 1 ( 0.0%) items with no interactions.


KeyboardInterrupt: 